# D-optimal experiment design: comparing ABPG and Frank-Wolfe
Solve the D-Optimal experiment design problem
$$
\begin{array}{ll}
\textrm{minimize}   & F(x):=\log\left(\det\left(\sum_{i=1}^n x_i V_i V_i^T\right)\right) \\
\textrm{subject to} & \sum_{i=1}^n x_i = 1, \\ 
                    & x_i\geq 0, \quad i=1,\ldots,n
\end{array}
$$
where $V_i\in R^m$ for $i=1,\ldots,n$.

Methods compared:
* Original Frank-Wolfe method
* Frank-Wolfe method with away steps
* Bregman Proximal Gradient (BPG) method with adaptive line search
* Accelerated Bregman Proximal Gradient (ABPG) method with gain adaption

In [37]:
cd  C:\\github\accbpg

[WinError 3] The system cannot find the path specified: 'C:\\\\github\\accbpg'
D:\projects\opts\accbpg\ipynb\ABPGvsFW


In [7]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 12, 'font.family': 'serif'})
#matplotlib.rcParams.update({'text.usetex': True})

import accbpg

In [18]:
n = 1000
m = [100, 200, 300, 400, 500, 600, 700, 800, 900]
K = 3
eps = ['1e-3', '1e-4', '1e-5', '1e-6', '1e-7', '1e-8']

Nmax = 20000
Nskip = int(Nmax/10)

Ieps = dict()
Teps = dict()
for s in eps:
    Ieps[s] = np.zeros((5, len(m), K))
    Teps[s] = np.zeros((5, len(m), K))

for i in range(len(m)):
    print("\n********** m = {0:d}, n = {1:d} **********".format(m[i], n))
    for k in range(K):
        f, h, L, x0Kh = accbpg.D_opt_design(m[i], n)
        x0KY = accbpg.D_opt_KYinit(f.H)
        x0Mx = (1-1e-3)*x0KY + 1e-3*x0Kh

        _, F_FWKY, _, _, T_FWKY = accbpg.D_opt_FW(f.H, x0Mx, 1e-8, maxitrs=Nmax, verbskip=Nskip)
        _, F_WAKY, _, _, T_WAKY = accbpg.D_opt_FW_away(f.H, x0Mx, 1e-8, maxitrs=Nmax, verbskip=Nskip)
        # _, F_WAKY_RS, _, _, T_WAKY_RS = accbpg.D_opt_FW_RS(f.H, h, L, x0Mx, 1e-8, gamma=2, maxitrs=Nmax, verbskip=Nskip)
        _, F_WAKY_RS, _, T_WAKY_RS  = accbpg.D_opt_FW_RS_adaptive(f, h, L, x0Mx, epsilon=1e-8, gamma=2, maxitrs=Nmax, linesearch=True, ls_ratio=2, verbskip=Nskip)
        _, F_LSKh, _, T_LSKh = accbpg.BPG(f, h, L, x0Mx, maxitrs=Nmax, linesearch=True, ls_ratio=2, verbskip=Nskip)
        _, F_ABKh, _, _, _, T_ABKh = accbpg.ABPG_gain(f, h, L, x0Mx, gamma=2, maxitrs=Nmax, ls_inc=2, ls_dec=2, restart=True, verbskip=Nskip)

        Fmin = min(F_FWKY.min(), F_WAKY.min(), F_WAKY_RS.min(), F_LSKh.min(), F_ABKh.min())
        F = [F_FWKY, F_WAKY, F_WAKY_RS, F_LSKh, F_ABKh]
        T = [T_FWKY, T_WAKY, T_WAKY_RS, T_LSKh, T_ABKh]
        for s in eps:
            for j in range(len(F)):
                I_eps = np.nonzero(F[j] - Fmin <= float(s))
                if len(I_eps[0]) > 0:
                    i_eps = I_eps[0][0]
                    t_eps = T[j][i_eps]
                else:
                    i_eps = Nmax + 1
                    t_eps = T[j][-1]
                Ieps[s][j,i,k] = i_eps
                Teps[s][j,i,k] = t_eps


********** m = 100, n = 1000 **********

Solving D-opt design problem using Frank-Wolfe method
     k      F(x)     pos_slack   neg_slack    time
     0   2.015e+01   1.848e+00   2.984e-01     0.0
  2000  -1.694e+00   1.702e-02   3.400e-01     0.2
  4000  -1.867e+00   1.014e-02   3.388e-01     0.4
  6000  -1.943e+00   7.644e-03   3.381e-01     0.5
  8000  -1.989e+00   6.085e-03   3.375e-01     0.7
 10000  -2.019e+00   5.166e-03   3.359e-01     0.9
 12000  -2.041e+00   4.375e-03   3.363e-01     1.1
 14000  -2.058e+00   3.928e-03   3.360e-01     1.3
 16000  -2.072e+00   3.451e-03   3.357e-01     1.5
 18000  -2.083e+00   3.163e-03   3.354e-01     1.7

Solving D-opt design problem using Frank-Wolfe method with away steps
     k      F(x)     pos_slack   neg_slack    time
     0   2.015e+01   1.848e+00   2.984e-01     0.0
  2000  -2.209e+00   1.256e-03   1.255e-03     0.4
  4000  -2.209e+00   5.161e-06   5.147e-06     0.7
  6000  -2.209e+00   3.912e-08   3.787e-08     1.1

FW RS adaptive m

KeyboardInterrupt: 

In [1]:
s = '1e-3'

m = np.array(m)
Igem = np.zeros((5,len(m)))
Imax = np.zeros((5,len(m)))
Imin = np.zeros((5,len(m)))
Tgem = np.zeros((5,len(m)))
Tmax = np.zeros((5,len(m)))
Tmin = np.zeros((5,len(m)))

for i in range(5):
    for j in range(len(m)):
        Igem[i,j] = Ieps[s][i,j].prod()**(1.0/K)
        Imax[i,j] = Ieps[s][i,j].max() 
        Imin[i,j] = Ieps[s][i,j].min()
        Tgem[i,j] = Teps[s][i,j].prod()**(1.0/K)
        Tmax[i,j] = Teps[s][i,j].max()
        Tmin[i,j] = Teps[s][i,j].min()

# Plot required number of iterations and time
plt.subplots(1, 2, figsize=(12, 4))
plt.subplots_adjust(wspace=0.3)

labels = [r"FW-KY", r"FW-away", r"FW-SR", r"BPG-LS", r"ABPG-g"]
linestyles = ['k:', 'g-', 'y-.', 'b-.', 'r--']

ax1 = plt.subplot(1,2,1)
for i in range(5):
    idx = np.nonzero(Igem[i] <= Nmax)[0]
    if len(idx) > 0:
        ax1.errorbar(m[idx], Igem[i,idx], yerr=[np.clip(Igem[i,idx]-Imin[i,idx], 0, None), Imax[i,idx]-Igem[i,idx]], 
                     fmt=linestyles[i], label=labels[i], marker='o', markersize=4, capsize=3)
ax1.legend()
ax1.set_yscale('log')
ax1.set_xlabel(r"$m$")
ax1.set_ylabel(r"Number of iterations")

ax2 = plt.subplot(1,2,2)
for i in range(5):
    idx = np.nonzero(Igem[i] <= Nmax)[0]
    if len(idx) > 0:
        ax2.errorbar(m[idx], Tgem[i,idx], yerr=[Tgem[i,idx]-Tmin[i,idx], Tmax[i,idx]-Tgem[i,idx]], 
                     fmt=linestyles[i], label=labels[i], marker='o', markersize=4, capsize=3)
ax2.legend()
ax2.set_yscale('log')
ax2.set_xlabel(r"m")
ax2.set_ylabel(r"Time (sec)")


NameError: name 'np' is not defined